# GAN / DCGAN

https://github.com/taewanme/DL_With_PyTorch/blob/master/Chapter07/GAN/DCGAN.ipynb

In [7]:
import os
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.utils as vutils
%matplotlib inline

In [10]:
if torch.cuda.is_available() :
    torch.cuda.empty_cache()

In [12]:
img_size = 64
batch_size = 64
lr = 0.0002
beta1 = 0.5
niter = 25
outf = 'output'

dataset = datasets.CIFAR10( root = 'data',
                          download = True,
                          transform = transforms.Compose([
                              transforms.Resize(img_size),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5),
                                                  (0.5, 0.5, 0.5))
                          ]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)

# size of latnet vector
nz = 100

# filter size of generator
ngf = 64

# filter size of discriminator
ndf = 64

# output image channels
nc = 3

14.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%

Extracting data\cifar-10-python.tar.gz to data


In [13]:
# 네트워크 초기화
# custom weights initialization called on netG and netD

def weights_init(m) :
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 :
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1 :
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [19]:
# 생성기
class _netG(nn.Module) :
    def __init__(self) :
        super(_netG, self).__init__()
        
        self.main = nn.Sequential(
        # input is Z, going into a convolution
        nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
        nn.BatchNorm2d(ngf*8),
        nn.ReLU(True),
        
        # state size, (ngf*8) X 4 X 4
        nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf*4),
        nn.ReLU(True),
            
        # state size, (ngf*4) X 8 X 8
        nn.ConvTranspose2d(ngf*4, ngf*2,  4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf*2),
        nn.ReLU(True),
            
        # state size, (ngf*2) X 16 X 16
        nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf),
        nn.ReLU(True),
            
        # state size, (ngf) X 32 X 32
        nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
        nn.Tanh()
            
        # state size, (nc) X 64 X 64    
        )
        
        
    def forward(self, input) :
        output = self.main(input)
        return output
    
netG = _netG()
netG.apply(weights_init)
print(netG)

_netG(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [14]:
# 판별기
class _netD(nn.Module) :
    def __init__(self) :
        super(_netD, self).__init__()
        
        self.main = nn.Sequential(
            # input : (nc) X 64 X 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size : (ndf) X 32 X 32

            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias = False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size : (ndf*2) X 16 X 16

            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size : (ndf*4) X 8 X 8

            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size : (ndf*8) X 4 X 4

            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, input) :
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)
    
netD = _netD()
netD.apply(weights_init)
print(netD)

_netD(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [16]:
# 오차 함수 정의

criterion = nn.BCELoss()
input = torch.FloatTensor(batch_size, 3, img_size, img_size)
noise = torch.FloatTensor(batch_size, nz, 1, 1)
fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)
label = torch.FloatTensor(batch_size)
real_label = 1
fake_label = 0

In [21]:
# 쿠다 
if torch.cuda.is_available():
    netD.cuda()
    netG.cuda()
    criterion.cuda()
    input, label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

In [20]:
# 옵티마이저 정의
fixed_noise = Variable(fixed_noise)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr, betas=(beta1, 0.999))

In [ ]:
# 학습

for epoch in range(niter) :
    for i, data in enumerate(dataloader, 0) :
        
        # (1) update D network : maximize log(D(x)) + log(1 - D(G(z)))
        
        # train with real
        netD.zero_grad() # 초기화
        real_cpu, _ = data
        batch_size = real_cpu.size(0)
        
        if torch.cuda.is_available() :
            real_cpu = real_cpu.cuda()
            
        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(batch_size).fill_(real_label)
        inputv = Variable(input)
        labelv = Variable(label)
        
        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()
        
        # train with fake
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        labelv = Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()
        
        
        # (2) update G network : maximize log(D(G(z)))
        
        netG.zero_grad()
        # fake labels = real for generator cost
        labelv = Variable(label.fill_(real_label))
        
        output = netD(fake)
        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()
        
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                      % (epoch, niter, i, len(dataloader),
                         errD.data.item(), errG.data.item(), D_x, D_G_z1, D_G_z2))
        
        if i % 100 == 0 :
            vutils.save_image(real_cpu,
                             '%s/real_samples.png' % outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03d.png' % (outf, epoch),
                    normalize=True)

[0/25][0/782] Loss_D: 0.9578 Loss_G: 4.4140 D(x): 0.5336 D(G(z)): 0.1804 / 0.0160
[0/25][1/782] Loss_D: 1.2188 Loss_G: 6.0705 D(x): 0.8494 D(G(z)): 0.5746 / 0.0034
[0/25][2/782] Loss_D: 0.8461 Loss_G: 6.8209 D(x): 0.7897 D(G(z)): 0.3624 / 0.0015
[0/25][3/782] Loss_D: 0.8357 Loss_G: 6.1731 D(x): 0.7158 D(G(z)): 0.2392 / 0.0027
[0/25][4/782] Loss_D: 1.1150 Loss_G: 7.3803 D(x): 0.7566 D(G(z)): 0.4446 / 0.0009
[0/25][5/782] Loss_D: 0.7785 Loss_G: 7.6152 D(x): 0.7365 D(G(z)): 0.2804 / 0.0007
[0/25][6/782] Loss_D: 0.9599 Loss_G: 7.4756 D(x): 0.6706 D(G(z)): 0.2838 / 0.0009
[0/25][7/782] Loss_D: 1.1000 Loss_G: 9.6017 D(x): 0.7419 D(G(z)): 0.4687 / 0.0001
[0/25][8/782] Loss_D: 0.8074 Loss_G: 7.7159 D(x): 0.6890 D(G(z)): 0.1276 / 0.0007
[0/25][9/782] Loss_D: 0.9070 Loss_G: 10.5527 D(x): 0.8186 D(G(z)): 0.4314 / 0.0000
[0/25][10/782] Loss_D: 0.5456 Loss_G: 8.0737 D(x): 0.7387 D(G(z)): 0.0838 / 0.0005
[0/25][11/782] Loss_D: 0.7674 Loss_G: 11.4271 D(x): 0.8741 D(G(z)): 0.4051 / 0.0000
[0/25][12/78

[0/25][99/782] Loss_D: 0.1769 Loss_G: 11.8623 D(x): 0.8946 D(G(z)): 0.0001 / 0.0000
[0/25][100/782] Loss_D: 0.1050 Loss_G: 6.0784 D(x): 0.9385 D(G(z)): 0.0058 / 0.0040
[0/25][101/782] Loss_D: 1.3949 Loss_G: 22.4244 D(x): 0.9634 D(G(z)): 0.6795 / 0.0000
[0/25][102/782] Loss_D: 0.7870 Loss_G: 21.0117 D(x): 0.6492 D(G(z)): 0.0000 / 0.0000
[0/25][103/782] Loss_D: 0.1187 Loss_G: 17.5968 D(x): 0.9032 D(G(z)): 0.0000 / 0.0000
[0/25][104/782] Loss_D: 0.0473 Loss_G: 11.3861 D(x): 0.9584 D(G(z)): 0.0000 / 0.0000
[0/25][105/782] Loss_D: 0.0618 Loss_G: 4.7735 D(x): 0.9667 D(G(z)): 0.0249 / 0.0151
[0/25][106/782] Loss_D: 1.6999 Loss_G: 17.6215 D(x): 0.9899 D(G(z)): 0.7631 / 0.0000
[0/25][107/782] Loss_D: 1.9784 Loss_G: 12.6935 D(x): 0.3180 D(G(z)): 0.0000 / 0.0000
[0/25][108/782] Loss_D: 0.3654 Loss_G: 7.8428 D(x): 0.7391 D(G(z)): 0.0004 / 0.0016
[0/25][109/782] Loss_D: 0.1286 Loss_G: 3.8829 D(x): 0.9453 D(G(z)): 0.0563 / 0.0348
[0/25][110/782] Loss_D: 0.7456 Loss_G: 8.6137 D(x): 0.9851 D(G(z)): 0.

[0/25][197/782] Loss_D: 0.9287 Loss_G: 2.9147 D(x): 0.5420 D(G(z)): 0.0434 / 0.0848
[0/25][198/782] Loss_D: 0.7937 Loss_G: 6.4595 D(x): 0.9019 D(G(z)): 0.4274 / 0.0038
[0/25][199/782] Loss_D: 0.3706 Loss_G: 5.3393 D(x): 0.8043 D(G(z)): 0.0666 / 0.0094
[0/25][200/782] Loss_D: 0.4017 Loss_G: 3.3708 D(x): 0.7981 D(G(z)): 0.0913 / 0.0542
[0/25][201/782] Loss_D: 0.7827 Loss_G: 6.4681 D(x): 0.8643 D(G(z)): 0.4121 / 0.0031
[0/25][202/782] Loss_D: 0.4130 Loss_G: 4.5774 D(x): 0.7499 D(G(z)): 0.0321 / 0.0156
[0/25][203/782] Loss_D: 0.2815 Loss_G: 3.2758 D(x): 0.8824 D(G(z)): 0.0908 / 0.0489
[0/25][204/782] Loss_D: 0.6954 Loss_G: 6.5589 D(x): 0.8976 D(G(z)): 0.3933 / 0.0027
[0/25][205/782] Loss_D: 0.6830 Loss_G: 3.8402 D(x): 0.6385 D(G(z)): 0.0257 / 0.0350
[0/25][206/782] Loss_D: 0.4407 Loss_G: 5.1651 D(x): 0.9643 D(G(z)): 0.2954 / 0.0100
[0/25][207/782] Loss_D: 0.3996 Loss_G: 4.2792 D(x): 0.7909 D(G(z)): 0.1000 / 0.0201
[0/25][208/782] Loss_D: 0.4996 Loss_G: 5.6234 D(x): 0.8845 D(G(z)): 0.2524 /

[0/25][295/782] Loss_D: 0.1101 Loss_G: 6.8814 D(x): 0.9591 D(G(z)): 0.0357 / 0.0012
[0/25][296/782] Loss_D: 0.0573 Loss_G: 6.8617 D(x): 0.9818 D(G(z)): 0.0369 / 0.0013
[0/25][297/782] Loss_D: 0.0724 Loss_G: 6.5981 D(x): 0.9818 D(G(z)): 0.0280 / 0.0016
[0/25][298/782] Loss_D: 0.0738 Loss_G: 6.5924 D(x): 0.9788 D(G(z)): 0.0354 / 0.0016
[0/25][299/782] Loss_D: 0.1100 Loss_G: 6.1251 D(x): 0.9493 D(G(z)): 0.0364 / 0.0026
[0/25][300/782] Loss_D: 0.0872 Loss_G: 7.2946 D(x): 0.9803 D(G(z)): 0.0603 / 0.0008
[0/25][301/782] Loss_D: 0.1462 Loss_G: 6.5561 D(x): 0.9324 D(G(z)): 0.0306 / 0.0015
[0/25][302/782] Loss_D: 0.1100 Loss_G: 7.0029 D(x): 0.9641 D(G(z)): 0.0566 / 0.0010
[0/25][303/782] Loss_D: 0.0831 Loss_G: 8.4516 D(x): 0.9821 D(G(z)): 0.0604 / 0.0003
[0/25][304/782] Loss_D: 0.0452 Loss_G: 7.6171 D(x): 0.9761 D(G(z)): 0.0197 / 0.0006
[0/25][305/782] Loss_D: 0.1830 Loss_G: 6.5336 D(x): 0.9431 D(G(z)): 0.0398 / 0.0019
[0/25][306/782] Loss_D: 0.1372 Loss_G: 9.2368 D(x): 0.9643 D(G(z)): 0.0902 /

[0/25][393/782] Loss_D: 0.7449 Loss_G: 3.7795 D(x): 0.7272 D(G(z)): 0.2453 / 0.0506
[0/25][394/782] Loss_D: 0.2184 Loss_G: 4.2070 D(x): 0.9578 D(G(z)): 0.1455 / 0.0263
[0/25][395/782] Loss_D: 0.3953 Loss_G: 3.9994 D(x): 0.8074 D(G(z)): 0.1259 / 0.0322
[0/25][396/782] Loss_D: 0.5356 Loss_G: 4.5169 D(x): 0.7911 D(G(z)): 0.2002 / 0.0253
[0/25][397/782] Loss_D: 0.3384 Loss_G: 4.4945 D(x): 0.8672 D(G(z)): 0.1495 / 0.0255
[0/25][398/782] Loss_D: 0.2741 Loss_G: 5.3797 D(x): 0.8959 D(G(z)): 0.1245 / 0.0108
[0/25][399/782] Loss_D: 0.4797 Loss_G: 2.6305 D(x): 0.7209 D(G(z)): 0.0841 / 0.1119
[0/25][400/782] Loss_D: 0.6802 Loss_G: 7.3262 D(x): 0.9248 D(G(z)): 0.3948 / 0.0014
[0/25][401/782] Loss_D: 1.2711 Loss_G: 2.0642 D(x): 0.4098 D(G(z)): 0.0098 / 0.2414
[0/25][402/782] Loss_D: 1.1027 Loss_G: 6.9992 D(x): 0.9654 D(G(z)): 0.5505 / 0.0023
[0/25][403/782] Loss_D: 0.4104 Loss_G: 5.1068 D(x): 0.7381 D(G(z)): 0.0270 / 0.0138
[0/25][404/782] Loss_D: 0.3383 Loss_G: 2.4879 D(x): 0.7934 D(G(z)): 0.0552 /

...

[1/25][592/782] Loss_D: 0.3122 Loss_G: 5.3091 D(x): 0.9449 D(G(z)): 0.1943 / 0.0083
[1/25][593/782] Loss_D: 0.5785 Loss_G: 2.7891 D(x): 0.7085 D(G(z)): 0.0918 / 0.0980
[1/25][594/782] Loss_D: 0.6021 Loss_G: 3.4957 D(x): 0.8444 D(G(z)): 0.2959 / 0.0518
[1/25][595/782] Loss_D: 0.7312 Loss_G: 3.8798 D(x): 0.7841 D(G(z)): 0.2964 / 0.0324
[1/25][596/782] Loss_D: 1.1389 Loss_G: 0.8811 D(x): 0.4518 D(G(z)): 0.0971 / 0.4766
[1/25][597/782] Loss_D: 1.3405 Loss_G: 5.6179 D(x): 0.9401 D(G(z)): 0.6637 / 0.0069
[1/25][598/782] Loss_D: 0.7835 Loss_G: 3.2576 D(x): 0.5633 D(G(z)): 0.0422 / 0.0719
[1/25][599/782] Loss_D: 1.1616 Loss_G: 1.3699 D(x): 0.5667 D(G(z)): 0.2770 / 0.3306
[1/25][600/782] Loss_D: 1.3587 Loss_G: 6.4119 D(x): 0.9319 D(G(z)): 0.6282 / 0.0038
[1/25][601/782] Loss_D: 1.5823 Loss_G: 2.2876 D(x): 0.3162 D(G(z)): 0.0147 / 0.1822
[1/25][602/782] Loss_D: 1.1482 Loss_G: 4.6262 D(x): 0.9038 D(G(z)): 0.5465 / 0.0243
[1/25][603/782] Loss_D: 0.7775 Loss_G: 2.4089 D(x): 0.6138 D(G(z)): 0.1384 /

[1/25][690/782] Loss_D: 0.7296 Loss_G: 2.3120 D(x): 0.6472 D(G(z)): 0.1168 / 0.1680
[1/25][691/782] Loss_D: 0.9622 Loss_G: 5.7602 D(x): 0.8578 D(G(z)): 0.4587 / 0.0079
[1/25][692/782] Loss_D: 1.0562 Loss_G: 2.6266 D(x): 0.5349 D(G(z)): 0.1316 / 0.1230
[1/25][693/782] Loss_D: 0.4875 Loss_G: 3.6467 D(x): 0.8779 D(G(z)): 0.2601 / 0.0447
[1/25][694/782] Loss_D: 0.5796 Loss_G: 5.3571 D(x): 0.8459 D(G(z)): 0.2778 / 0.0082
[1/25][695/782] Loss_D: 0.4063 Loss_G: 3.3892 D(x): 0.7533 D(G(z)): 0.0383 / 0.0565
[1/25][696/782] Loss_D: 0.4426 Loss_G: 3.8418 D(x): 0.8639 D(G(z)): 0.2245 / 0.0318
[1/25][697/782] Loss_D: 0.3287 Loss_G: 3.6327 D(x): 0.8401 D(G(z)): 0.1082 / 0.0457
[1/25][698/782] Loss_D: 0.5161 Loss_G: 3.1254 D(x): 0.7971 D(G(z)): 0.1955 / 0.0634
[1/25][699/782] Loss_D: 0.4744 Loss_G: 5.0315 D(x): 0.8857 D(G(z)): 0.2467 / 0.0114
[1/25][700/782] Loss_D: 0.3224 Loss_G: 3.7911 D(x): 0.7836 D(G(z)): 0.0308 / 0.0485
[1/25][701/782] Loss_D: 0.4131 Loss_G: 3.0193 D(x): 0.8439 D(G(z)): 0.1775 /

[2/25][6/782] Loss_D: 2.5642 Loss_G: 1.5573 D(x): 0.1412 D(G(z)): 0.0034 / 0.3248
[2/25][7/782] Loss_D: 1.0681 Loss_G: 4.8462 D(x): 0.9095 D(G(z)): 0.5207 / 0.0139
[2/25][8/782] Loss_D: 0.7097 Loss_G: 0.5318 D(x): 0.6038 D(G(z)): 0.0495 / 0.6525
[2/25][9/782] Loss_D: 2.3965 Loss_G: 8.2774 D(x): 0.9846 D(G(z)): 0.8412 / 0.0006
[2/25][10/782] Loss_D: 3.5410 Loss_G: 3.1134 D(x): 0.0842 D(G(z)): 0.0018 / 0.1099
[2/25][11/782] Loss_D: 0.5364 Loss_G: 2.7073 D(x): 0.9376 D(G(z)): 0.3109 / 0.1094
[2/25][12/782] Loss_D: 0.7784 Loss_G: 5.4766 D(x): 0.9029 D(G(z)): 0.4192 / 0.0082
[2/25][13/782] Loss_D: 1.3984 Loss_G: 1.1584 D(x): 0.3902 D(G(z)): 0.0923 / 0.4045
[2/25][14/782] Loss_D: 0.9653 Loss_G: 4.3378 D(x): 0.9451 D(G(z)): 0.5157 / 0.0205
[2/25][15/782] Loss_D: 0.4310 Loss_G: 3.9207 D(x): 0.7855 D(G(z)): 0.1179 / 0.0304
[2/25][16/782] Loss_D: 0.4609 Loss_G: 2.2672 D(x): 0.7303 D(G(z)): 0.0792 / 0.1291
[2/25][17/782] Loss_D: 0.5552 Loss_G: 3.9693 D(x): 0.9205 D(G(z)): 0.3396 / 0.0261
[2/25][1

[2/25][105/782] Loss_D: 0.6442 Loss_G: 2.6467 D(x): 0.6893 D(G(z)): 0.1602 / 0.1007
[2/25][106/782] Loss_D: 0.9720 Loss_G: 6.2716 D(x): 0.8471 D(G(z)): 0.4784 / 0.0032
[2/25][107/782] Loss_D: 1.2750 Loss_G: 1.9176 D(x): 0.4052 D(G(z)): 0.0268 / 0.2429
[2/25][108/782] Loss_D: 0.6225 Loss_G: 3.9271 D(x): 0.9195 D(G(z)): 0.3493 / 0.0298
[2/25][109/782] Loss_D: 0.3192 Loss_G: 4.7870 D(x): 0.9086 D(G(z)): 0.1653 / 0.0153
[2/25][110/782] Loss_D: 0.4448 Loss_G: 2.8353 D(x): 0.7320 D(G(z)): 0.0657 / 0.0854
[2/25][111/782] Loss_D: 0.5714 Loss_G: 4.5952 D(x): 0.8976 D(G(z)): 0.3222 / 0.0160
[2/25][112/782] Loss_D: 0.4237 Loss_G: 3.3680 D(x): 0.7503 D(G(z)): 0.0602 / 0.0568
[2/25][113/782] Loss_D: 0.6150 Loss_G: 3.5483 D(x): 0.8029 D(G(z)): 0.2717 / 0.0381
[2/25][114/782] Loss_D: 0.5030 Loss_G: 4.1112 D(x): 0.8131 D(G(z)): 0.2127 / 0.0213
[2/25][115/782] Loss_D: 0.4809 Loss_G: 2.4522 D(x): 0.7393 D(G(z)): 0.1098 / 0.1245
[2/25][116/782] Loss_D: 0.8274 Loss_G: 5.7869 D(x): 0.8927 D(G(z)): 0.4559 /

[2/25][203/782] Loss_D: 0.8024 Loss_G: 3.4311 D(x): 0.6507 D(G(z)): 0.2014 / 0.0495
[2/25][204/782] Loss_D: 0.9463 Loss_G: 4.2772 D(x): 0.7853 D(G(z)): 0.4013 / 0.0226
[2/25][205/782] Loss_D: 0.7492 Loss_G: 2.3190 D(x): 0.6035 D(G(z)): 0.0828 / 0.1438
[2/25][206/782] Loss_D: 0.9530 Loss_G: 3.6394 D(x): 0.7514 D(G(z)): 0.4085 / 0.0465
[2/25][207/782] Loss_D: 0.5670 Loss_G: 3.7492 D(x): 0.7703 D(G(z)): 0.1774 / 0.0411
[2/25][208/782] Loss_D: 0.4547 Loss_G: 2.8212 D(x): 0.7713 D(G(z)): 0.1129 / 0.1130
[2/25][209/782] Loss_D: 0.8512 Loss_G: 4.0452 D(x): 0.7852 D(G(z)): 0.3797 / 0.0268
[2/25][210/782] Loss_D: 0.4115 Loss_G: 3.3532 D(x): 0.7614 D(G(z)): 0.0776 / 0.0541
[2/25][211/782] Loss_D: 0.3810 Loss_G: 3.7554 D(x): 0.9128 D(G(z)): 0.1981 / 0.0417
[2/25][212/782] Loss_D: 0.5036 Loss_G: 3.0447 D(x): 0.7680 D(G(z)): 0.1539 / 0.0689
[2/25][213/782] Loss_D: 0.4802 Loss_G: 3.2421 D(x): 0.8296 D(G(z)): 0.2125 / 0.0562


In [ ]:
!ls -al output/

In [ ]:
Image.open('output/real_samples.png')

In [ ]:
Image.open('output/fake_samples_epoch_024.png')